### Autograds

Cada tensor tiene un parámetro booleano ```requires_grad``` que nos permitirá excluir o incluir la construcción de gráficos de dependencia para hacer el seguimiento de las gradientes de nuestro algoritmos.  Si alguno de los tensores necesita ser diferenciado, entonces tendremos que activar este parámetro antes. El poder activar y desactivar gradientes en ciertos tensores nos permite "congelar" ciertos layers en las redes neuronales, para poder re-entrenar un modelo (fine-tuning).

Tal como se mencionó antes, nos sirve para crear gráficos de dependecia pero ¿Por qué esto es necesario? Cuando hacemos la optimización de nuestro algoritmo descenso de la gradiente, es necesario **tener un registro de las gradientes**, es decir, hacer un seguimiento de los errores a través de la red.

Entonces, recordemos que en una red neuronal pasan dos cosas: 

- Forward Propagation: La red neuronal hace su mejor predicción y pasa todos los datos del input a través de las neuronas y capas que contenga con el fin de hacer su mejor predicción.

- Backward Propagation: La red neuronal propaga el error a través de las derivadas y debe de tener una colección de ellas (date cuenta en este caso es necesita el ``requires_grad`` activo) optimizando los parámetros usando el descenso de la gradiente. En este sentido, torch.autograd es una herramienta para computar vectorialmente una matriz producto Jacobiana.

In [1]:
import torch

a = torch.tensor([2.,3.], requires_grad = True) 
b = torch.tensor([6.,4.], requires_grad = True)

Q = 3*a**3 - b**2 #Asumiendo que a y b son parámetros de una red neuronal

#donde dQ/da = [36,81] y dQ/db = [-12,-8]

In [2]:
print('Q-Vector', Q)
print('a.grad=',a.grad)
print('b.grad=',b.grad)

Q-Vector tensor([-12.,  65.], grad_fn=<SubBackward0>)
a.grad= None
b.grad= None


Cada vez que llamamos ```.backward()``` derivamos nuestra función con respecto a todos los vectores implicados que tengan el ```requires_grad``` activo. Esto nos sirve principalmente para poder hacer luego el seguimiento de estas gradientes junto con el optimizador. Las gradientes se van a almacenar en ```tensor.grad``` donde tensor es el nombre del tensor. El optimizador llamará a estos vectores almacenados y hará su cálculo a través de ellos. Nota que por eso mismo podremos hacer el seguimiento de los parámetros. En este caso no tenemos optimizador. 

In [3]:
vector_gradiente_inicial = torch.tensor([1.,1.]) #Inicializo mi vector de gradientes

In [4]:
#Calculamos las gradientes y las almacenamos en los vectores activos.
Q.backward(gradient=vector_gradiente_inicial) 

In [5]:
print('Q-Vector', Q)
print('a.grad=',a.grad) #La derivada o "gradiente" de Q con respecto a a
print('b.grad=',b.grad) #dQ/da

Q-Vector tensor([-12.,  65.], grad_fn=<SubBackward0>)
a.grad= tensor([36., 81.])
b.grad= tensor([-12.,  -8.])


#### Actualizando los parámetros como en una red neuronal:

Hasta este punto tenemos una forma de cómo almacenar las gradientes según una función. Pero cómo esto nos va a ser útil para **actualizar los parámetros**. Supongamos que tenemos una función ```Y = W*x + b``` donde ``W`` son los pesos a actualizar, ``X`` son los inputs y ``b`` es el bias o la constante de la ecuacion.

In [6]:
W = torch.randn(10,1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
x = torch.rand(1, 10)

Ahora nos toca formalizar la función ```Y = W*x + b``` recodermos acá que mientras los tensores W y b tengan el parámetro ``requires_grad`` activado, también lo tendrá la formalización de Y. Esto lo vemos si esque invocamos a Y y vemos que tiene asociada la función ``<AddBackward0>`` asociada.

In [7]:
Y = torch.matmul(x, W) + b
Y

tensor([[1.4971]], grad_fn=<AddBackward0>)

Necesitamos una función de pérdida para poder optimizar sobre. Digamos que esta función está dada por ```loss_function = (1 - Y)/2 ``` Esto nos servirá para poder calcular las gradientes de la función de pérdida con respecto a Y, es decir: dloss/dY. Que esto terminará dándonos la pérdida con respecto a Y y con respecto a W, según la regla de la cadena. Así, indirectamente tendremos qué W's de Y contribuyen más a la función de pérdida.

In [8]:
loss = (1 - Y)/2

In [9]:
#Calculamos las gradientes de loss con respecto a Y.
loss.backward()

In [10]:
print('Dloss/W', W.grad)
print('Dloss/b', W.grad)
print('W', W)

Dloss/W tensor([[-0.3478],
        [-0.2626],
        [-0.2391],
        [-0.1834],
        [-0.2414],
        [-0.1846],
        [-0.1108],
        [-0.0056],
        [-0.1630],
        [-0.4082]])
Dloss/b tensor([[-0.3478],
        [-0.2626],
        [-0.2391],
        [-0.1834],
        [-0.2414],
        [-0.1846],
        [-0.1108],
        [-0.0056],
        [-0.1630],
        [-0.4082]])
W tensor([[ 0.5150],
        [-1.1924],
        [-1.7766],
        [ 1.8514],
        [-0.4424],
        [-0.3146],
        [ 1.1133],
        [-2.6996],
        [-0.2121],
        [ 1.0215]], requires_grad=True)


Ahora que tenemos las gradientes almacenadas, procedemos a **desactivarlas temporalmente** para poder actualizarlas y luego limpiar la "caché" de gradientes puesto que ya no nos sirven las antiguas. Esto lo hacemos con ```torch.no_grad()``` y usualmente se hace en cada iteración.

In [11]:
W #Tenemos los Weights iniciales

tensor([[ 0.5150],
        [-1.1924],
        [-1.7766],
        [ 1.8514],
        [-0.4424],
        [-0.3146],
        [ 1.1133],
        [-2.6996],
        [-0.2121],
        [ 1.0215]], requires_grad=True)

In [12]:
with torch.no_grad(): #Desactivamos las gradientes
    W = W - 0.1 * W.grad

W #Los weights finales

tensor([[ 0.5497],
        [-1.1662],
        [-1.7527],
        [ 1.8697],
        [-0.4183],
        [-0.2961],
        [ 1.1244],
        [-2.6991],
        [-0.1958],
        [ 1.0623]])

# RE-ESTRUCTURAR!
#### ¿Cómo usar estos conceptos para re-entrenar un modelo?

In [13]:
from mlp_mnist_model import MLP #Es necesario tener la estructura del algoritmo en un archivo aparte.

#Cargamos nuestro modelo
model = torch.load('mnist_model.pt')

#Cargamos datos generados de manera aleatoria (Solo para actualizar los parámetros)
data = torch.rand(64, 784) #Tendré 64 muestras de un tensor de 784 (lo q acepta mi red)
labels = torch.rand(64, 10) #Los labels con probabilidades

In [19]:
#La estructura del modelo:


In [20]:
#Podemos ver los parámetros del modelo (Los weights)


In [ ]:
#Predecimos para ver qué tanto nuestro algoritmo clasifica el ruido
output = model(data)
pred = torch.argmax(output, dim=1)

#Vamos a definir la pérdida (loss) de la sgte manera
loss = (output - labels).sum()

In [ ]:
loss.backward()

In [ ]:
#Definimos un optimizador
optim = torch.optim.SGD(model.parameters(), lr= 1e-2, momentum= 0.9)
#Inicializamos el descenso de la gradiente
optim.step()

In [ ]:
#El orden es opt.zero_grad(), loss.backward(), opt.step()